In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

path = "/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/raw/weather_raw.csv"
weather_raw = pd.read_csv(path)
path = "/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/raw/accidents_raw.csv"
accidents_raw = pd.read_csv(path)
path = "/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/raw/traffic_raw.csv"
traffic_raw = pd.read_csv(path)

/tmp/ipython-input-3717684131.py:6: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  accidents_raw = pd.read_csv(path)


In [46]:
#Elimina espacios
def clean_string_columns(df):
    for col in df.select_dtypes(include="object").columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
        )
    return df

#Normalizacion de nulos
def normalize_nulls(df):
    import numpy as np
    null_equivalents = [
        "", " ", "  ", "NULL", "null", "None", "none",
        "N/A", "n/a", "NA", "na", "-", "--"
    ]
    return df.replace(null_equivalents, np.nan)

#Relleno de nulos
def fill_numeric_nulls(df, fill_value=0):
    num_cols = df.select_dtypes(include=["int64", "float64"]).columns
    df[num_cols] = df[num_cols].fillna(fill_value)
    return df

#Dar formato de fecha
def normalize_date_columns(df):
    import pandas as pd

    date_keywords = ['date', 'fecha', 'datetime']

    for col in df.columns:
        if any(keyword in col.lower() for keyword in date_keywords):
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
            )
            df[col] = pd.to_datetime(
                df[col],
                errors='coerce',
                dayfirst=True
            )

    return df

#Normalizacion de campo de fecha para traffic_transformed
traffic_transformed = traffic_raw['date'] = pd.to_datetime({
    'year': traffic_raw['Yr'],
    'month': traffic_raw['M'],
    'day': traffic_raw['D']
}, errors='coerce')

#Conversion de Fechas para weather
import pandas as pd
for col in weather_raw.columns:
    if "date" in col.lower():
        weather_raw[col] = pd.to_datetime(weather_raw[col], errors="coerce").dt.strftime("%d/%m/%Y")

In [48]:
def clean_transformed(df):
    df = clean_string_columns(df)
    df = normalize_nulls(df)
    df = fill_numeric_nulls(df)
    df = normalize_date_columns(df)
    return df

In [ ]:
weather_transformed = clean_transformed(weather_raw)
accidents_transformed = clean_transformed(accidents_raw)
traffic_transformed = clean_transformed(traffic_raw)


In [ ]:
weather_transformed.to_parquet("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/weather_transformed.parquet", index=False)
accidents_transformed.to_parquet("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/accidents_transformed.parquet", index=False)
traffic_transformed.to_parquet("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/traffic_transformed.parquet", index=False)

In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/weather_transformed.parquet")
files.download("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/accidents_transformed.parquet")
files.download("/content/drive/MyDrive/Colab_Notebooks/urban_mobility_project/transformed/traffic_transformed.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>